## 2.4. Przykład - chat z pamięcią

In [ ]:
# Instalujemy tylko to, czego potrzebujemy do chatu z pamięcią.
!pip install -q langchain-openai python-dotenv langchain-core


In [ ]:
import os
from dotenv import load_dotenv

# 1) Wczytujemy zmienne środowiskowe z .env (w tym OPENAI_API_KEY)
load_dotenv()

# 2) Inicjalizujemy model czatowy od OpenAI przez adapter langchain-openai
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o-mini",  # możesz podmienić na inny wspierany model
    temperature=0         # 0 = maksymalna przewidywalność, dobre do testów pamięci
)

print("Model gotowy.")


In [ ]:
# Użyjemy ChatPromptTemplate do zdefiniowania "roli" i wejścia użytkownika.
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# SYSTEM: instrukcje dla modelu (styl, rola)
# USER:   wiadomość użytkownika (podamy ją później jako {input})
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Prowadzisz przyjazną rozmowę po polsku i pamiętasz kontekst."),
    ("user", "{input}")
])

# LCEL (LangChain Expression Language): prompt -> model -> parser
# Na razie to "goły" łańcuch bez pamięci.
base_chain = chat_prompt | llm | StrOutputParser()

print("Łańcuch bazowy gotowy (bez pamięci).")


In [ ]:
# Do pamięci użyjemy wbudowanej klasy InMemoryChatMessageHistory
# oraz wrappera RunnableWithMessageHistory, który "wstrzykuje"
# historię do promptu na podstawie session_id.

from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# "store" będzie prostą mapą: session_id -> historia wiadomości
store = {}

def get_history(session_id: str) -> InMemoryChatMessageHistory:
    """Zwraca (lub tworzy) obiekt historii dla danej sesji."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Opakowujemy nasz bazowy łańcuch w "łańcuch z pamięcią".
# - input_messages_key: nazwa pola z nowym wejściem użytkownika
# - history_messages_key: nazwa pola, pod którą wstrzykiwana jest historia
chain_with_memory = RunnableWithMessageHistory(
    base_chain,
    get_history,                     # funkcja pobierająca historię po session_id
    input_messages_key="input",      # ta sama nazwa co w prompt {input}
    history_messages_key="history",  # LangChain wstrzyknie historię do promptu
)

print("Łańcuch z pamięcią gotowy.")


In [ ]:
# Każda "konwersacja" ma swój session_id.
# Dzięki temu możesz równolegle utrzymywać wiele niezależnych czatów.
sid = "demo-session-1"

# 1) Użytkownik przedstawia się
resp1 = chain_with_memory.invoke(
    {"input": "Cześć! Nazywam się Michał i lubię programować w Pythonie."},
    config={"configurable": {"session_id": sid}}
)
print("Bot:", resp1)

# 2) Pytanie zależne od kontekstu (model powinien "pamiętać" imię)
resp2 = chain_with_memory.invoke(
    {"input": "Jak mam na imię i jaki język programowania lubię?"},
    config={"configurable": {"session_id": sid}}
)
print("Bot:", resp2)

# 3) Doprecyzowanie (pamięć + bieżący kontekst)
resp3 = chain_with_memory.invoke(
    {"input": "Zgadza się. Dodaj, że prowadzę kurs o LangChain."},
    config={"configurable": {"session_id": sid}}
)
print("Bot:", resp3)
e

In [ ]:
# Nowe session_id = nowa, pusta pamięć
sid2 = "demo-session-2"

# Tu model nie powinien pamiętać, kim jest Michał z poprzedniej sesji.
respA = chain_with_memory.invoke(
    {"input": "Kim jestem i co lubię programować?"},
    config={"configurable": {"session_id": sid2}}
)
print("Bot (sesja 2, bez kontekstu):", respA)

# Dodajemy informację w tej sesji...
respB = chain_with_memory.invoke(
    {"input": "Mam na imię Ania i lubię Javę."},
    config={"configurable": {"session_id": sid2}}
)
print("Bot:", respB)

# ...i sprawdzamy pamięć tylko w tej sesji:
respC = chain_with_memory.invoke(
    {"input": "Jak mam na imię i jaki język lubię?"},
    config={"configurable": {"session_id": sid2}}
)
print("Bot:", respC)


In [ ]:
# Ta funkcja pozwala "popykać" w konsoli bez przełączania komórek.
# Wywołuj: chat("demo-session-1", "Twoja wiadomość")
def chat(session_id: str, user_message: str):
    """Wyślij wiadomość do bota w ramach wskazanej sesji."""
    reply = chain_with_memory.invoke(
        {"input": user_message},
        config={"configurable": {"session_id": session_id}}
    )
    print(f"[You @{session_id}]: {user_message}")
    print(f"[Bot]: {reply}")

# Przykład:
# chat("demo-session-1", "Przypomnij, co mówiłem o Pythonie.")


InMemoryChatMessageHistory trzyma listę wiadomości (user/assistant) dla danego session_id.\
RunnableWithMessageHistory:\
Przy każdym wywołaniu pobiera historię z get_history(session_id).\
Dokłada ją do promptu (klucz history), zanim model wygeneruje odpowiedź.\
Po wygenerowaniu odpowiedzi automatycznie dopisuje ją do historii.\
Dzięki temu kolejne wywołania w ramach tej samej sesji widzą wcześniejszy kontekst.\
Różne session_id ⇒ izolowane pamięci (oddzielne rozmowy).\
Chcesz wersję z podsumowującą pamięcią (automatyczne streszczenie starej historii, żeby oszczędzać tokeny), albo wariant, który serializuje historię do\ pliku/Redis? Mogę dorzucić gotowe komórki.\